In [3]:
!git clone https://github.com/Gurupatil0003/Chest-X-Ray-Dataset

fatal: destination path 'Chest-X-Ray-Dataset' already exists and is not an empty directory.


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
img_size=64
batch_size=32

In [6]:
train_path="/content/Chest-X-Ray-Dataset/chest_xray/train"
test_path="/content/Chest-X-Ray-Dataset/chest_xray/test"

In [7]:
train_data=ImageDataGenerator(rescale=1.0/255).flow_from_directory(
    train_path,
    target_size=(img_size,img_size),
    batch_size=batch_size,
    class_mode="binary",
    )
train_data

Found 693 images belonging to 2 classes.


In [8]:
test_data=ImageDataGenerator(rescale=1.0/255).flow_from_directory(
    test_path,
    target_size=(img_size,img_size),
    batch_size=batch_size,
    class_mode="binary",
    )
test_data

Found 333 images belonging to 2 classes.


In [63]:
model=Sequential([
    Conv2D(64,(3,3),input_shape=(img_size,img_size,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [64]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [65]:
model.fit(train_data,epochs=7,validation_data=test_data)

Epoch 1/7
22/22 ━━━━━━━━━━━━━━━━━━━━ 22s 897ms/step - accuracy: 0.8076 - loss: 0.3567 - val_accuracy: 0.5946 - val_loss: 0.8913
Epoch 2/7
22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 782ms/step - accuracy: 0.9134 - loss: 0.2904 - val_accuracy: 0.5946 - val_loss: 1.3962
Epoch 3/7
22/22 ━━━━━━━━━━━━━━━━━━━━ 22s 855ms/step - accuracy: 0.9208 - loss: 0.2502 - val_accuracy: 0.5946 - val_loss: 0.8191
Epoch 4/7
22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 782ms/step - accuracy: 0.9328 - loss: 0.1865 - val_accuracy: 0.5946 - val_loss: 0.4921
Epoch 5/7
22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 788ms/step - accuracy: 0.9204 - loss: 0.1732 - val_accuracy: 0.5946 - val_loss: 0.9430
Epoch 6/7
22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 793ms/step - accuracy: 0.9158 - loss: 0.1101 - val_accuracy: 0.6517 - val_loss: 0.9200
Epoch 7/7
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 797ms/step - accuracy: 0.9512 - loss: 0.0944 - val_accuracy: 0.6787 - val_loss: 0.9091


In [66]:
model.save("chest_xray_model.h5")

In [67]:
pip install gradio

In [68]:
import gradio as gr
import numpy as np

img_size=64
cls_name=["Normal","Pneumonia"]

def predict(img):
  img=img.resize((img_size,img_size))
  img=np.array(img)/255.0
  img=np.expand_dims(img, axis=0)
  prediction=model.predict(img)[0][0]

  label=1 if prediction >= 0.5 else 0
  confidence=prediction if prediction >= 0.5 else 1-prediction
  return f"{cls_name[label]} (confidence:{confidence})"


In [61]:
# import gradio as gr
# import numpy as np
# img_size=64
# class_name=["Normal","Pnemounia"]

# def predict(image):
#   image =image.resize((img_size,img_size))
#   image=np.array(image)/255
#   image=np.expand_dims(image,axis=0)
#   prediction=model.predict(image)

#   return class_name[int(prediction[0][0])]

In [69]:
interface=gr.Interface(
    fn=predict,
    inputs=gr.Image(type='pil'),
    outputs=gr.Textbox(label="Prediction")
)
interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://15a5b41e3bfb4fc061.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
